In [1]:
import sys; sys.path.append('../../src/helpers')
import ast
import csv
from data_manipulation import custom_transformers as ct
import gensim.downloader as api
import numpy as np
import pandas as pd
import pickle
from sklearn.pipeline import Pipeline 


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nathanfranklin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nathanfranklin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2019-06-12 22:03:21,292 summarizer.preprocessing.cleaner INFO     'pattern' package not found; tag filters are not available for English


In [2]:
with open('../../models/RFC_deployment_model.pkl', 'rb') as file:  
    rfc = pickle.load(file)

In [3]:
user_input_plot_summary = '''This story is about a boy named Jim. 
Jim decided he wanted to be a pirate, but not just any pirate. No Jim
 wanted to be aQWDWQND space pirate. someone who had laser guns and could take his family across the universe
 and fight aliens 238 and protect his homeland from the evil aliens who were invading
 the solar XNIAMne system that he lived in. But d18x1m ncziomz nnwiMQ the year 19272672 far in the future nothin could have prepared jim for the fights and battles
 and stars and explosions and bombs and explodings that were coming his way.
 You will have to read more to find out what happened to Jim!'''

In [4]:
user_input_plot_summary

'This story is about a boy named Jim. \nJim decided he wanted to be a pirate, but not just any pirate. No Jim\n wanted to be aQWDWQND space pirate. someone who had laser guns and could take his family across the universe\n and fight aliens 238 and protect his homeland from the evil aliens who were invading\n the solar XNIAMne system that he lived in. But d18x1m ncziomz nnwiMQ the year 19272672 far in the future nothin could have prepared jim for the fights and battles\n and stars and explosions and bombs and explodings that were coming his way.\n You will have to read more to find out what happened to Jim!'

In [5]:
new_entry_df = pd.DataFrame(data=[user_input_plot_summary], columns=['plotSum'])

In [6]:
new_entry_df

,plotSum
0,This story is about a boy named Jim. \nJim dec...


In [7]:
word2vec_model = api.load("glove-wiki-gigaword-50")

2019-06-12 22:03:24,017 gensim.models.utils_any2vec INFO     loading projection weights from /Users/nathanfranklin/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
/anaconda3/envs/AVC/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-06-12 22:03:58,794 gensim.models.utils_any2vec INFO     loaded (400000, 50) matrix from /Users/nathanfranklin/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz


In [8]:
with open('../../models/LDA_model.pkl', 'rb') as file:  
    lda_model = pickle.load(file)
    
with open('../../models/tf_vect.pkl', 'rb') as file:  
    tf_vect = pickle.load(file)

In [9]:
steps = [('tokenizer', ct.Tokenizer()),
                ('filterer', ct.Filter_sentence(filter_names=True)),
                ('mean_words2vec_er',ct.MeanEmbeddingVectorizer(word2vec_model)),
                ('pruner', ct.StemmingLemming(prune_type='Porter')),
                ('joiner_transformer', ct.Joiner())]
         
pipeline = Pipeline(steps)
cleaned_summary_w_w2vec = pipeline.transform(new_entry_df)

2019-06-12 22:03:58,894 data_manipulation.custom_transformers INFO     Calling the Tokenizer transformer
2019-06-12 22:03:58,919 data_manipulation.custom_transformers INFO     Calling the filter sentence transformer
2019-06-12 22:03:58,946 data_manipulation.custom_transformers INFO     Calling the StemmingLemming transformer
2019-06-12 22:03:58,954 data_manipulation.custom_transformers INFO     Calling the Joiner transformer


In [10]:
cleaned_summary_w_w2vec

,plotSum,plotSum2vec
0,stori boy name jim jim decid want pirat pirat ...,"[0.32892950944471777, -0.04371063072201714, 0...."


In [11]:
lda_transformer = ct.LDA_Vectorizer(lda_model, tf_vect)

In [12]:
prediction_DF = lda_transformer.transform(cleaned_summary_w_w2vec)

2019-06-12 22:03:59,021 data_manipulation.custom_transformers INFO     Calling the LDA transformer


In [13]:
prediction_DF

,plotSum,plotSum2vec,plotSumLDA
0,stori boy name jim jim decid want pirat pirat ...,"[0.32892950944471777, -0.04371063072201714, 0....","[[0.0010000000373910525, 0.0010000000214915638..."


In [14]:
plotSumLDA_list = prediction_DF['plotSumLDA'].values[0][0]
plotSumLDA_DF = pd.DataFrame([plotSumLDA_list])

In [15]:
plotSumVecList = prediction_DF['plotSum2vec'].values[0]
plotSumVecs_DF = pd.DataFrame([plotSumVecList])

In [16]:
print(plotSumLDA_DF.shape)
print(plotSumVecs_DF.shape)

(1, 20)
(1, 50)


In [17]:
newColsWord2vec = []
for columnIndex in range(len(plotSumVecs_DF.columns)):
    newColsWord2vec.append('word2vec_'+str(columnIndex+1))
plotSumVecs_DF.columns = newColsWord2vec

newColsLDA = []
for columnIndex in range(len(plotSumLDA_DF.columns)):
    newColsLDA.append('LDA_'+str(columnIndex+1))
plotSumLDA_DF.columns = newColsLDA

X = pd.concat([plotSumLDA_DF,plotSumVecs_DF], axis=1)

In [18]:
X

,LDA_1,LDA_2,LDA_3,LDA_4,LDA_5,LDA_6,LDA_7,LDA_8,LDA_9,LDA_10,...,word2vec_41,word2vec_42,word2vec_43,word2vec_44,word2vec_45,word2vec_46,word2vec_47,word2vec_48,word2vec_49,word2vec_50
0,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.270824,0.189065,0.171577,...,-0.141457,0.15074,0.046191,0.11809,0.095982,0.064091,-0.150027,-0.263996,-0.146851,0.083161


In [19]:
predicted_probs = rfc.predict_proba(X)

In [20]:
list(predicted_probs[0])

[0.08,
 0.0,
 0.0,
 0.24,
 0.05,
 0.07,
 0.0,
 0.22,
 0.31,
 0.0,
 0.05,
 0.01,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.54,
 0.42,
 0.0,
 0.06,
 0.0,
 0.0,
 0.0,
 0.18]

In [21]:
with open('../../models/genre_names.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)[0]

In [22]:
your_list

['adventure',
 'anthol_biog_autobiog',
 'chick_lit',
 'children',
 'comedy',
 'crime_fiction',
 'existential_philosophy',
 'fantasy',
 'fiction',
 'historical_fiction',
 'horror',
 'informational',
 'lgbt',
 'non_fiction_lit',
 'realistic_fiction',
 'religious',
 'romance',
 'science_fiction',
 'speculative_fiction',
 'sports',
 'suspense/thriller/spy',
 'true_crime',
 'war',
 'western',
 'young_adult']

In [23]:
from scoring import score_user_input